In [1]:
import cv2
import math
import numpy as np

from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error as mse

## Auxiliary Functions

In [2]:
def showImage(image, name):
    cv2.imshow(name, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
def threshold(pixel, threshold = 0):
    if np.any(pixel != threshold):
        return True
    else:
        return False

Convolution Filters

In [4]:
sobelX = np.array((
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]), dtype="int")
sobelX = sobelX*(1/8)

sobelY = np.array((
    [-1, -2, -1],
    [0, 0, 0],
    [1, 2, 1]), dtype="int")
sobelY = sobelY*(1/8)

sobelXY = "sobelXY"

bilinear = np.array((
    [1, 2, 1],
    [2, 4, 2],
    [1, 2, 1]), dtype="int")
bilinear = bilinear*(1/16)

gaussian = np.array((
    [1,4,6,4,1],
    [4,16,24,16,4],
    [6,24,36,24,6],
    [4,16,24,16,4],
    [1,4,6,4,1]), dtype="int")
gaussian = gaussian*(1/256)

In [5]:
def convolRoi(roi, kernel):
    
    if "sobelXY" in kernel:
        convolved = convolRoiXY(roi, kernel)
    else:
        convolved = cv2.filter2D(roi, -1, kernel)
    
    return convolved

In [6]:
def convolRoiXY(roi, kernel):
    
    convX = convolRoi(roi, sobelX)
    convY = convolRoi(roi, sobelY)
    convolved = np.add(np.absolute(convX),np.absolute(convY))
    
    return convolved

In [7]:
def closeImg(img, kernel):
    closeImg = cv2.dilate(img, kernel, iterations=5)
    closeImg = cv2.erode(closeImg, kernel, iterations=5)
    
    return closeImg

## Functions

In [8]:
def ExtractObject(S2, ObjectMask):
    
    height, width, channel = S2.shape
    ExtractedObject = np.zeros((height,width,3), np.uint8)
    
    for i, row in enumerate(ObjectMask):
        for j, p in enumerate(row):
            if threshold(p):
                ExtractedObject[i,j] = S2[i,j]
    
    return ExtractedObject

In [9]:
def ApplyFilter(ExtractedObject, FilterIndex):
    
    if FilterIndex == 1:
        FilteredObject = convolRoi(ExtractedObject, sobelXY)
        pass
    elif FilterIndex == 2:
        FilteredObject = convolRoi(ExtractedObject, bilinear)
        pass
    elif FilterIndex == 3:
        FilteredObject = convolRoi(ExtractedObject, gaussian)
        pass
    else:
        FilteredObject = ExtractedObject
        pass
    
    return FilteredObject

In [10]:
def ObjectBlender(S1, FilteredObject, alpha = 0.5):
    beta = (1.0 - alpha)
    
    BlendingResult = cv2.addWeighted(FilteredObject, alpha, S1, beta, 0.0)
    return BlendingResult

In [11]:
def CompareResult(BlendingResult, S2, metric):
    
    if metric == 1:
        # Signal to noise ratio, units: decibals
        ErrorValue = cv2.PSNR(S2,BlendingResult)
        print("PSNR: ", ErrorValue,"dB")
        pass
    elif metric == 2:
        # mean structural similarity index over the image, unit: index between 0 & 1
        ErrorValue = ssim(S2, BlendingResult, multichannel=True)
        print("SSIM: ", ErrorValue)
        pass
    elif metric == 3:
        # mean-squared error between two images
        ErrorValue = mse(S2, BlendingResult)
        print("MSE: ", ErrorValue)
        pass
    else:
        ErrorValue = "No Metric Specified"
        pass
    
    return ErrorValue

#### Stage 2

In [12]:
def RemoveGreen(img):
    
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    lower_green = np.array([36, 25, 25])
    upper_green = np.array([86, 255,255])
 
    # preparing the mask to overlay
    mask = cv2.inRange(hsv, lower_green, upper_green)
    showImage(mask, "mask")
    
    imgNoGreen = np.copy(img)
    imgNoGreen[mask != 0] = [0, 0, 0]
    
    return imgNoGreen

In [13]:
def NewBackground(imgNoBg, newBackground):
    
    height, width, channel = imgNoBg.shape
    
    newBackground = cv2.resize(newBackground, (width, height), interpolation = cv2.INTER_AREA)
    imgNewBG = np.copy(newBackground)

    for i, row in enumerate(imgNoBg):
        for j, p in enumerate(row):
            if threshold(p):
                imgNewBG[i,j] = imgNoBg[i,j]
    
    return imgNewBG

## Runner

#### Stage 1

In [14]:
S1 = cv2.imread("Part 2 - Multiple Objects/footwear_no/1_colour.jpeg", 1)
S2 = cv2.imread("Part 2 - Multiple Objects/footwear_no/3_colour.jpeg", 1)
ObjectMask = cv2.imread("Part 2 - Multiple Objects/footwear_no/masks/footware_no_3_colour_mask_3_mask.png", 1)

In [15]:
extractedObject = ExtractObject(S2, ObjectMask)
showImage(extractedObject, "Extracted Image")

In [16]:
filteredObject = ApplyFilter(extractedObject, 0)
showImage(filteredObject, "Filtered Image")

In [17]:
blendedObject = ObjectBlender(S1, filteredObject)
showImage(blendedObject, "Blended Image")

In [18]:
for metric in range(1,4):
    errorValue = CompareResult(blendedObject, S2, metric)

PSNR:  17.583762055130162 dB
SSIM:  0.6957159318487717
MSE:  1134.2379011140047


#### Stage 2

In [19]:
S1noGreen = RemoveGreen(S1)
showImage(S1noGreen, "S1 No green")

S2noGreen = RemoveGreen(S2)
showImage(S2noGreen, "S2 No green")

In [20]:
newBackground = cv2.imread("backgrounds/cartoon_bg.jpg", 1)

S1NewBg = NewBackground(S1noGreen, newBackground)
showImage(S1NewBg, "S1 New Background")

S2NewBg = NewBackground(S2noGreen, newBackground)
showImage(S2NewBg, "S2 New Background")